In [21]:
# csv문서 기반 QA
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq 

In [22]:
# 1. 데이터 로드
from pathlib import Path
file_path = Path("backend") / "data" / "raw" / "for_vectorDB" / "housing_vector_data.csv"
loader = CSVLoader(file_path=str(file_path), encoding='utf-8-sig')

In [23]:
data = loader.load()
data[0]

Document(metadata={'source': 'backend\\data\\raw\\for_vectorDB\\housing_vector_data.csv', 'row': 0}, page_content='주택명: 오늘공동체주택\n지번주소: 서울특별시 도봉구 도봉동 351-2\n도로명주소: 서울특별시 도봉구 도봉로191길 80 (도봉동)\n시군구: 도봉구\n동명: 도봉동\n태그: 테마:인증통과 육아형 취미형, 지하철:도봉산역 도봉산역, 자격요건:제한없음, 마트:세븐일레븐 도봉산길점, 병원:부부약국, 학교:도봉초등학교, 시설:김근태기념도서관, 카페:유영정원')

In [24]:
data[10]

Document(metadata={'source': 'backend\\data\\raw\\for_vectorDB\\housing_vector_data.csv', 'row': 10}, page_content='주택명: 더클래식하우스\n지번주소: 서울특별시 서대문구 연희동 703-15 더클래식 하우스\n도로명주소: 서울특별시 서대문구 홍연2길 16 (연희동, 더클래식 하우스)\n시군구: 서대문구\n동명: 연희동\n태그: 테마:인증통과 예술형, 지하철:홍대입구역, 자격요건:제한없음, 마트:GS/CU/ 홈마트, 병원:세브란스/ 동신병원, 학교:홍대 연세대 명지대 서연중 연희초, 시설:서대문구청/ 공원, 카페:더클래식/ 본솔')

In [25]:
print(data[0].page_content)

주택명: 오늘공동체주택
지번주소: 서울특별시 도봉구 도봉동 351-2
도로명주소: 서울특별시 도봉구 도봉로191길 80 (도봉동)
시군구: 도봉구
동명: 도봉동
태그: 테마:인증통과 육아형 취미형, 지하철:도봉산역 도봉산역, 자격요건:제한없음, 마트:세븐일레븐 도봉산길점, 병원:부부약국, 학교:도봉초등학교, 시설:김근태기념도서관, 카페:유영정원


In [26]:
# 2. 데이터 분할
# 문서를 특정 기준(청크)로 분할
# RecursiveCharacterTextSplitter : 다양한 문자로 재귀적으로 분할하여 효과적인 문자를 찾습니다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

split_documents = text_splitter.split_documents(data)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 130


In [27]:
print("===== PAGE 2 =====")
print(split_documents[2])
print("===== PAGE 3 =====")
print(split_documents[3])

===== PAGE 2 =====
page_content='주택명: 연희동 청년연구자 주택 애스트리 23
지번주소: 서울특별시 서대문구 연희동 48-66 *연희23
도로명주소: 서울특별시 서대문구 연희로18길 36 (연희동, *연희23)
시군구: 서대문구
동명: 연희동
태그: 테마:인증통과 청년형, 자격요건:제한없음' metadata={'source': 'backend\\data\\raw\\for_vectorDB\\housing_vector_data.csv', 'row': 2}
===== PAGE 3 =====
page_content='주택명: 비투코하우징홍은
지번주소: 서울특별시 서대문구 홍은동 414-14
도로명주소: 서울특별시 서대문구 증가로4길 57 (홍은동)
시군구: 서대문구
동명: 홍은동
태그: 테마:인증통과 청년형 취미형, 지하철:홍제역, 자격요건:제한없음, 마트:CU., 병원:세브란스병원, 학교:명지대학교' metadata={'source': 'backend\\data\\raw\\for_vectorDB\\housing_vector_data.csv', 'row': 3}


In [28]:
# 3. 임베딩 생성
# GroqEmbeddings는 없음. 다른 임베딩 사용하여야 함

# 한국어 임베딩 모델 사용
embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sbert-nli",  # 한국어 특화 모델
    model_kwargs={'device': 'cpu'},   # CPU 사용 (GPU 없어도 됨)
    encode_kwargs={'normalize_embeddings': True}  # 정규화-코사인 유사도 계산 최적화, 백터크기차이 최소화, 검색 속도 향상
)

print("🔄 한국어 임베딩 모델 로딩 중... (시간이 걸릴 수 있습니다)")

🔄 한국어 임베딩 모델 로딩 중... (시간이 걸릴 수 있습니다)


In [29]:
# 4. DB생성 및 저장
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="practicing_vectorstore"
)

## 2. RAG 수행

In [31]:
# 5. retriever 생성

# 문서 검색 함수
def get_documents(query):
    if "추천" in query or "어떤" in query:
        # MMR로 다양한 옵션 제공
        retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": 5, # 검색 결과 상위 5개 반환
                "fetch_k": 15, # MMR 알고리즘이 고려할 후보 수 
                "lambda_mult": 0.9} # 유사성90% + 다양성10% 비율
        )
    else:
        # Similarity로 정확한 매칭
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    
    return retriever.invoke(query)

retriever_runnable = RunnableLambda(get_documents)


# 테스트: 검색기에 쿼리를 날려 검색된 chunk를 반환
result = get_documents("강서구")

# 검색된 결과를 출력
len(result), print(result[0].page_content)

주택명: 코이노니아 스테이
지번주소: 서울특별시 강서구 내발산동 681-1 코이노니아 스테이
도로명주소: 서울특별시 강서구 강서로47길 60 (내발산동, 코이노니아 스테이)
시군구: 강서구
동명: 내발산동
태그: 테마:인증통과 청년형 신혼형, 지하철:우장산역 마곡역 발산역, 자격요건:제한없음, 마트:CU 세븐일레븐 등 다수, 병원:이대서울병원 미즈메디병원 기타(의료특구) 다수, 학교:명덕고 명덕외고 덕원여고 덕원예고 화곡고 등 다수, 시설:서울시립도서관(예정) 수명산작은도서관, 카페:스타벅스 이디아 등 다수


(5, None)

In [32]:
# prompt 템플릿 생성
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 청년주택 추천 전문가입니다."),
    ("human", "사용자 질문: {query}\n검색 결과: {context}\n위 정보를 바탕으로 답변해주세요.")
])

# 결과: 메시지 객체들의 리스트
messages = prompt.format_messages(query="강남 청년주택", context="검색결과...")
print(messages)

[SystemMessage(content='당신은 청년주택 추천 전문가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='사용자 질문: 강남 청년주택\n검색 결과: 검색결과...\n위 정보를 바탕으로 답변해주세요.', additional_kwargs={}, response_metadata={})]


In [33]:
# 7. 모델 생성
import os
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(model_name="gemma2-9b-it", api_key=os.getenv("GROQ_API_KEY"), temperature=0.7)

In [35]:
# 8. 체인 생성
chain = (
    {"context": retriever_runnable, "query": RunnablePassthrough()} # 입력구조정의
    # retriever_runnable의 실행결과를 context에 전달하고 query는 사용자의 질문을 전달
    | prompt
    | llm
    | StrOutputParser() # 출력 파싱
)



## 체인 실행

In [ ]:
import time

query = "강서구 주택 추천해줘"

# 일반 실행
print("=== 일반 체인 실행 ===")
response = chain.invoke(query)
print(response)

# 스트리밍 실행 (LLM 응답을 실시간으로 출력, 속도 조절)
print("\n=== 스트리밍 체인 실행 ===")
for chunk in chain.stream(query):
    if chunk:  # chunk가 비어있지 않은 경우만
        print(chunk, end="", flush=True)
        time.sleep(0.03)  # 0.03초 지연으로 출력 속도 조절

=== 일반 체인 실행 ===
강서구 주택을 찾으시는군요! 

검색 결과, **'코이노니아 스테이'**라는 주택을 추천드립니다. 

* **강서구 내발산동**에 위치해 있어 강서구에서의 생활이 편리하겠죠? 
* **'테마:인증통과 청년형 신혼형'**으로  청년층에게 특화된 주택입니다.
* **'지하철:우장산역 마곡역 발산역'** 에 가까우므로 교통 접근성 또한 좋습니다. 

다른 정보들이 필요하시면 언제든지 물어보세요! 😊  




=== 스트리밍 체인 실행 ===
강서구 주택을 찾으시는군요! 검색결과에 **코이노니아 스테이**라는 주택이 나왔습니다. 

**코이노니아 스테이**는 강서구 내발산동에 위치하고 있으며, 다음과 같은 특징이 있습니다.

* **테마:** 인증통과 청년형 신혼형 
* **교통:** 지하철 우장산역, 마곡역, 발산역 가까이
* **편의시설:** CU, 세븐일레븐 등 다수의 마트, 이대서울병원, 미즈메디병원 등 다수의 병원, 명덕고, 명덕외고, 덕원여고, 덕원예고, 화곡고 등 다수의 학교, 서울시립도서관(예정), 수명산작은도서관, 스타벅스, 이디아 등 다수의 카페가 인근에 위치해있습니다.
* **자격요건:** 제한없음

좀 더 자세한 정보를 원하신다면, 주택명을 검색해보세요. 


